In [3]:
import tweepy
import keyring
import datetime
import pandas as pd
import numpy as np
import pickle

## Parameters and imports

In [2]:
days_to_post = 30
# only works for 1 or 2 posts per day, since only the date of the most recent post is checked in
# chapter 4
posts_per_day = 2
pics_per_day = 2
texts_per_day = 1
#starting_date = datetime.date(2022, 5, 9)
starting_date = datetime.date.today()
user_id_dog = 1483397937787023362
user_id_cat = 0

In [6]:
with open('../content/dogs/posts.pickle','rb') as f:
    dogs = pickle.load(f)

## Twitter Authentification

In [9]:
client = tweepy.Client(keyring.get_password('twitter','bearer_token'), keyring.get_password('twitter','api_key'), keyring.get_password('twitter','api_key_secret'), keyring.get_password('twitter','access_token'), keyring.get_password('twitter','access_token_secret'))        

In [10]:
auth = tweepy.OAuthHandler(keyring.get_password('twitter','api_key'), keyring.get_password('twitter','api_key_secret'))
auth.set_access_token(keyring.get_password('twitter','access_token'), keyring.get_password('twitter','access_token_secret'))
api = tweepy.API(auth)

## Check if this day has already been posted

In [25]:
# use twitter api to return the last tweet from dog account
check = api.user_timeline(user_id=user_id_dog, count=1)

In [26]:
# check if dog account has already tweeted today
if check == []:
    post_of_day = 1
elif check[0].created_at.date() != datetime.date.today():
    post_of_day = 1
else:
    post_of_day = 2

## Post

In [27]:
# retrieve text and picture of tweet from dataframe
dog_post = dogs.loc[(dogs.date == datetime.date.today()) & (dogs.post_of_day == post_of_day)]
dog_text = dog_post.text.item()
dog_pic = dog_post.pics.item()

In [18]:
if dog_text == None:
    dog_pic = dogs.loc[(dogs.date == datetime.date.today()) & (dogs.post_of_day == post_of_day)]['pics'].item()
    media_dog = api.media_upload(dog_pic)
    client.create_tweet(media_ids=[media_dog.media_id])
else:
    dog_pic = dogs.loc[(dogs.date == datetime.date.today()) & (dogs.post_of_day == post_of_day)]['pics'].item()
    media_dog = api.media_upload(dog_pic)
    client.create_tweet(text=dog_text, media_ids=[media_dog.media_id])